In [3]:
import pandas as pd
import numpy as np
import sqlalchemy as sqla
from io import StringIO
from collections import Counter

#establish connection to database
conn = sqla.create_engine('sqlite:///yelp.sqlite')

#split business.json into bussines, attributes, hours and store in a database 
business=pd.read_json("yelp_dataset/yelp_academic_dataset_business.json",lines=True)
business.set_index("business_id",inplace=True)

attributes=pd.DataFrame.from_dict(business["attributes"].dropna().to_dict(),orient="index")
attributes.index.name="business_id"

hours=pd.DataFrame.from_dict(business["hours"].dropna().to_dict(),orient="index")
hours.index.name="business_id"

business.drop(["hours","attributes"],axis=1,inplace=True)
business.replace("",np.nan,inplace=True)

#simplify category column by using the most frequent term for each row
business.reset_index(inpalce=True)
business.fillna({"categories":"Not Avaliable"},inplace=True)
business["categories"]=business.categories.str.replace(" ","")
category=business.categories.str.split(",")
category=category.apply(Counter)
combined_category=Counter()
for c in category:
    combined_category.update(c)
cate_count=pd.DataFrame.from_dict(combined_category,"index").reset_index().sort_values(0,ascending=False)
cate_count.columns=["name","count"]
simple_category=[]
for row in business.categories.str.split(","):
    for category in cate_count["name"]:
        if category in row:
            simple_category.append(category)
            break
business.categories=pd.Series(simple_category)
business.set_index("business_id",inplace=True)

#store in sql
business.to_sql('business',conn,if_exists="replace")
hours.to_sql('hours',conn,if_exists="replace")
attributes.to_sql('attributes',conn,if_exists="replace")

#convert checkin.json into tidy dataframe and store in sql
checkins=pd.read_json("yelp_dataset/yelp_academic_dataset_checkin.json",lines=True,orient="record",chunksize=10000)
for checkin in checkins:
    checkin.set_index("business_id",inplace=True)
    checkin=pd.DataFrame.from_dict(checkin["time"].to_dict(),orient="index")
    checkin.reset_index(inplace=True)
    checkin=checkin.melt(id_vars="index")
    days=pd.read_table(StringIO(checkin['variable'].to_csv(None,index=None)),sep='-',header=None)
    checkin=checkin.join(days)
    checkin.drop("variable",axis=1,inplace=True)
    checkin=checkin[["index",0,1,"value"]]
    checkin.columns=["business_id","weekday","hour","checkins"]
    checkin.set_index("business_id",inplace=True)
    checkin["weekday"]=checkin["weekday"].astype("category")
    checkin.dropna(inplace=True)
    checkin.to_sql('checkin',conn,if_exists="append")

#store review in sql
reviews=pd.read_json("yelp_dataset/yelp_academic_dataset_review.json",chunksize=100000,lines=True)
for review in reviews:
    review.set_index("business_id",inplace=True)
    review.to_sql("review",conn,if_exists="append")